

**Three Factor Model in Crypto Market** Group 11, ISOM3350, Option2

Downloading the necessry libraries:



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data as pdr
import zipfile
import statsmodels.api as s
import requests, zipfile, io
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import warnings
warnings.filterwarnings('ignore')

**Step 1**: Collecting data from [Kaggle](https://www.kaggle.com/datasets/sudalairajkumar/cryptocurrency-historical-prices-coingecko?select=avalanche-2.csv). We put it on our [Github](https://github.com/bobbobr/fintech/blob/main/SMB.zip?raw=true), so it is easier to access.

**Step 2**: Making the date the same format. Originally we had 50 csv files and the date foemat varied, so we make all dates YYYY-MM-DD.


In [ ]:
http_response = urlopen("https://github.com/bobbobr/fintech/blob/main/SMB.zip?raw=true")
with zipfile.ZipFile(BytesIO(http_response.read())) as zip_ref:
    file_list = zip_ref.namelist()
    dataframes = []
    #open zip while for getting data
    for file_name in file_list:
        with zip_ref.open(file_name) as csv_file:
            dataframe = pd.read_csv(csv_file)
            dataframe[file_name] = file_name
            #replace .csv from file_name 
            dataframe[file_name] = dataframe[file_name].str.replace(".csv",'')
            dataframe=dataframe.drop(columns=[file_name])
            # calculating log return and shifting data on 1 day
            dataframe["return"] = (dataframe.price.pct_change()+1).apply(np.log)
            dataframe["return"] = dataframe["return"].shift(1)
            dataframe["total_volume"] = dataframe["total_volume"].shift(1)
            dataframe["market_cap"] = dataframe["market_cap"].shift(1)
            dataframes.append(dataframe)
df=pd.concat(dataframes)
df=df.sort_values(by=['date'])
# some data manipulation to make index as datetime
df.groupby(['date']).count()
df.index = pd.to_datetime(df['date'])
df=df.drop(columns=['date'])
# calculating return of market portfolio, we will use it later for MKT factor
find = df.groupby(['date'])['return'].mean()
# from this data we will have al least 26 unique tokens for calculations 
start = pd.to_datetime("2019-07-03")
df = df.loc[start:]
find = pd.DataFrame(find)
find = find.loc[start:]
# df is our main dataframe with all data 
df

,price,total_volume,market_cap,coin_name,return
date,,,,,
2019-07-03,1.010494,1.128160e+07,7.359924e+08,tezos,-0.014073
2019-07-03,1.546498,3.250984e+05,0.000000e+00,terra-luna,-0.013355
2019-07-03,0.007617,7.299095e+07,4.508715e+08,vechain,-0.007445
2019-07-03,1.728825,6.039301e+06,1.747015e+09,leo-token,-0.008116
2019-07-03,5.343843,1.127086e+08,1.319056e+09,cosmos,0.003346
...,...,...,...,...,...
2023-05-16,313.937586,4.269176e+08,4.922638e+10,binancecoin,0.002771
2023-05-16,1820.044760,4.914907e+09,2.212054e+11,ethereum,0.001006
2023-05-16,0.909948,6.188990e+06,8.912784e+08,theta-token,-0.011765


**Step 3**: building the SMB factor

In [ ]:
# define function for calculation SMB factor 
def smb(data,df):
  # take data and look at dataframe about this day 
  df = df.loc[data]
  # sorting by market cap
  df = df.sort_values(by = "market_cap")
  n = len(df)
  # We will take 30% as a small and 30% as a big 
  sm=int(0.3*n)
  # take an average 
  s = np.mean(df.iloc[:sm]['return'])
  b = np.mean(df.iloc[n-sm:]['return'])
  # return our factor small minus big 
  return(s-b)


In [ ]:
sm = []
# We need to calculate for all days 
for i in df.index.unique():
  # call our function smb and save this factor in a list 
  sm.append(smb(i,df))
# create DataFrame with factor SMB for each day
sm = pd.DataFrame(sm, index =df.index.unique(), columns = ["SMB"])
sm

,SMB
date,
2019-07-03,-0.021261
2019-07-04,-0.029252
2019-07-05,-0.068342
2019-07-06,0.056596
2019-07-07,0.060402
...,...
2023-05-12,-0.010183
2023-05-13,-0.015998
2023-05-14,-0.003374


**Step 4**: building the BTC Lag Factor

In [ ]:
# btc lag 
# We uploaded data about BTC price in github 
url1 = "https://github.com/bobbobr/fintech/blob/main/bitcoin.csv?raw=true"
btc = pd.read_csv(url1, index_col= ["date"])
btc.index = pd.to_datetime(btc.index)
# calculate log return and shift our data by 1 day
btc["BTC_lag"] = (btc.price.pct_change()+1).apply(np.log)
btc["BTC_lag"] = btc["BTC_lag"].shift(1)
# slice only from the necessary date 
btc = btc.loc[start:]
btc= btc["BTC_lag"].iloc[:]
btc = pd.DataFrame(btc)
# our factor about BTC_lag
btc

,BTC_lag
date,
2019-07-03,-0.023139
2019-07-04,0.017121
2019-07-05,0.101223
2019-07-06,-0.064321
2019-07-07,-0.021818
...,...
2023-05-12,0.001171
2023-05-13,-0.022501
2023-05-14,-0.008811


**Step 5**: building the market factor, we considered the risk-free rate [the staking rewards on USDT](https://www.stakingrewards.com/earn/tether/)

In [ ]:
market = []
# calcule daily risk-free from annualized yearly return
rf=(1.0497)**(1/365)-1
rf
for i in df.index.unique():
    # calculate for each day market return from the market portfolio and substracting risk-free from it
    df1 = find.loc[i]
    bt1 = btc.loc[i]
    market.append(np.mean(df1['return']+bt1['BTC_lag'])-rf)
market = pd.DataFrame(market, index =df.index.unique(), columns = ["Market"])
market

,Market
date,
2019-07-03,-0.017808
2019-07-04,-0.008229
2019-07-05,0.121545
2019-07-06,-0.075944
2019-07-07,-0.001093
...,...
2023-05-12,0.007438
2023-05-13,-0.056679
2023-05-14,0.007280


**Step 6**: adding the factors to the data frame for each coin

In [ ]:
# Getting data from my github 
http_response = urlopen("https://github.com/bobbobr/fintech/blob/main/SMB.zip?raw=true")
with zipfile.ZipFile(BytesIO(http_response.read())) as zip_ref:
    file_list = zip_ref.namelist()
    dataframes = []
    # Let's try our model on this token
    cosmos_file_name = "cosmos.csv"
    
    if cosmos_file_name in file_list:
        with zip_ref.open(cosmos_file_name) as csv_file:
            dataframe = pd.read_csv(csv_file)
            dataframe["cosmos"] = cosmos_file_name
            # Again calculate log return 
            dataframe["return"] = (dataframe.price.pct_change() + 1).apply(np.log)
            dataframes.append(dataframe)
cosmos = pd.DataFrame(dataframes[0])
cosmos.index = pd.to_datetime(cosmos['date'])

cosmos = cosmos.drop(columns=["date",'cosmos'])
cosmos = cosmos.loc[start:]
# Our data frame with information about this token
cosmos

,price,total_volume,market_cap,coin_name,return
date,,,,,
2019-07-03,5.343843,1.488661e+08,1.287511e+09,cosmos,-0.025118
2019-07-04,5.629593,1.255823e+08,1.357531e+09,cosmos,0.052092
2019-07-05,5.510555,9.882670e+07,1.329365e+09,cosmos,-0.021372
2019-07-06,5.378548,9.527893e+07,1.295066e+09,cosmos,-0.024247
2019-07-07,5.320921,8.122304e+07,1.283973e+09,cosmos,-0.010772
...,...,...,...,...,...
2023-05-12,11.049896,1.590692e+08,3.235297e+09,cosmos,0.004066
2023-05-13,11.174650,1.715892e+08,3.269392e+09,cosmos,0.011227
2023-05-14,10.889997,9.343409e+07,3.194248e+09,cosmos,-0.025803


In [ ]:
cosmos = cosmos.merge(market, on='date')
cosmos = cosmos.merge(btc, on='date')
cosmos = cosmos.merge(sm, on='date')
# Add our calculated factors to cosmos dataframe 
cosmos

,price,total_volume,market_cap,coin_name,return,Market,BTC_lag,SMB
date,,,,,,,,
2019-07-03,5.343843,1.488661e+08,1.287511e+09,cosmos,-0.025118,-0.017808,-0.023139,-0.021261
2019-07-04,5.629593,1.255823e+08,1.357531e+09,cosmos,0.052092,-0.008229,0.017121,-0.029252
2019-07-05,5.510555,9.882670e+07,1.329365e+09,cosmos,-0.021372,0.121545,0.101223,-0.068342
2019-07-06,5.378548,9.527893e+07,1.295066e+09,cosmos,-0.024247,-0.075944,-0.064321,0.056596
2019-07-07,5.320921,8.122304e+07,1.283973e+09,cosmos,-0.010772,-0.001093,-0.021818,0.060402
...,...,...,...,...,...,...,...,...
2023-05-12,11.049896,1.590692e+08,3.235297e+09,cosmos,0.004066,0.007438,0.001171,-0.010183
2023-05-13,11.174650,1.715892e+08,3.269392e+09,cosmos,0.011227,-0.056679,-0.022501,-0.015998
2023-05-14,10.889997,9.343409e+07,3.194248e+09,cosmos,-0.025803,0.007280,-0.008811,-0.003374


**Step 8**: building the regression models

In [ ]:
# Define X for regression (independent variable)
X = cosmos[['Market','SMB', 'BTC_lag']]
# Define Y for regression (devendent variable)
y = cosmos["return"]
# Add intercept in our model 
X = s.add_constant(X)
# Fitting model
ff_model = s.OLS(y, X).fit()
# Look at the result 
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     3.023
Date:                Wed, 17 May 2023   Prob (F-statistic):             0.0287
Time:                        13:10:29   Log-Likelihood:                 1851.4
No. Observations:                1414   AIC:                            -3695.
Df Residuals:                    1410   BIC:                            -3674.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.002      0.299      0.7

In [ ]:
# Again we did the same, but for each cryptocurrencies from our dataset 
http_response = urlopen("https://github.com/bobbobr/fintech/blob/main/SMB.zip?raw=true")
with zipfile.ZipFile(BytesIO(http_response.read())) as zip_ref:
    file_list = zip_ref.namelist()
    dataframes = []
    for i in file_list:
      cosmos_file_name = i
    
      if cosmos_file_name in file_list:
          with zip_ref.open(cosmos_file_name) as csv_file:
              dataframe = pd.read_csv(csv_file)
              dataframe["cosmos"] = cosmos_file_name
              dataframe["return"] = (dataframe.price.pct_change() + 1).apply(np.log)
              
      cosmos = pd.DataFrame(dataframe)
      cosmos.index = pd.to_datetime(cosmos['date'])

      cosmos = cosmos.drop(columns=["date",'cosmos'])
      cosmos = cosmos.loc[start:]
      cosmos = cosmos.merge(market, on='date')
      cosmos = cosmos.merge(btc, on='date')
      cosmos = cosmos.merge(sm, on='date')
      X = cosmos[['Market','SMB', 'BTC_lag']]
      y = cosmos["return"]
      X = s.add_constant(X)
      ff_model = s.OLS(y, X).fit()
      print("This is for " + str(cosmos_file_name))
      print(ff_model.summary())
      intercept, b1, b2, b3 = ff_model.params

This is for algorand.csv
                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     3.115
Date:                Wed, 17 May 2023   Prob (F-statistic):             0.0253
Time:                        13:11:02   Log-Likelihood:                 1870.9
No. Observations:                1414   AIC:                            -3734.
Df Residuals:                    1410   BIC:                            -3713.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0014      

In [ ]:
# Adding token from out-of-sample, in order to look at results and do everything the same
url = "https://github.com/bobbobr/fintech/blob/main/zil.csv?raw=true"
zil= pd.read_csv(url, index_col= ["Date"])
zil.index = pd.to_datetime(zil.index)
zil = zil.sort_index()
zil["return"] = (zil.Close.pct_change()+1).apply(np.log)
zil = zil.loc[start:]
zil = zil["return"].iloc[:]
zil = pd.DataFrame(zil)
zil

,return
Date,
2019-07-03,-0.008552
2019-07-04,-0.047104
2019-07-05,0.005771
2019-07-06,0.070356
2019-07-07,0.025884
...,...
2023-05-11,-0.047668
2023-05-12,0.016564
2023-05-13,-0.007611


In [ ]:
# Define our model and slice to [:-1], because I downloaded it 05/16 and didn't have informatiom about this day 
X = cosmos[['Market','SMB', 'BTC_lag']][:-1]
y = zil["return"]
X = s.add_constant(X)
ff_model = s.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     3.201
Date:                Wed, 17 May 2023   Prob (F-statistic):             0.0226
Time:                        13:11:33   Log-Likelihood:                 1733.8
No. Observations:                1413   AIC:                            -3460.
Df Residuals:                    1409   BIC:                            -3439.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.738e-06      0.002     -0.003      0.9

In [ ]:
# Everything the same, but for ATOM token and look at the results. Also slice it 
url = "https://github.com/bobbobr/fintech/blob/main/Binance_ATOMUSDT_daily.csv?raw=true"
atom= pd.read_csv(url, index_col= ["Date"])
atom.index = pd.to_datetime(atom.index)
atom = atom.sort_index()
atom["return"] = (atom.Close.pct_change()+1).apply(np.log)
atom = atom.loc[start:]
atom = atom["return"].iloc[:]
atom = pd.DataFrame(atom)
X = cosmos[['Market','SMB', 'BTC_lag']][:-1]
y = atom["return"]
X = s.add_constant(X)
ff_model = s.OLS(y, X).fit()
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                 return   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.955
Date:                Wed, 17 May 2023   Prob (F-statistic):              0.119
Time:                        13:11:51   Log-Likelihood:                 1831.0
No. Observations:                1413   AIC:                            -3654.
Df Residuals:                    1409   BIC:                            -3633.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.002      0.145      0.8